In [1]:
import json
import os
import numpy as np
import re
from tqdm import tqdm
from glob import glob
from collections import defaultdict
from datasets import Audio
import torchaudio
import torch
import soundfile as sf

In [2]:
sr = 16000
reader = Audio(sampling_rate = sr)

In [3]:
data = []
with open('dataset-whisper-stt-2023-06-19.jsonl') as fopen:
    for l in tqdm(fopen):
        data.append(json.loads(l))

7210208it [00:22, 317181.58it/s]


In [4]:
len(data)

7210208

In [5]:
d = defaultdict(list)
for d_ in data:
    if d_['avg_logprob'] >= -0.31:
        k = f"{d_['no']}-{d_['i']}"
        d[k].append(d_)
        
len(d)

239079

In [6]:
def read(f):
    audio = reader.decode_example(reader.encode_example(f))['array']
    return audio, len(audio) / sr

In [7]:
!mkdir /home/husein/ssd3/postfilter-stt
!rm -rf /home/husein/ssd3/postfilter-stt/part1
!mkdir /home/husein/ssd3/postfilter-stt/part1

mkdir: cannot create directory ‘/home/husein/ssd3/postfilter-stt’: File exists


In [8]:
import string
printable = set(string.printable)

In [9]:
import torchaudio
import torch
import os

In [ ]:
maxlen = 12

index = 0
directory = '/home/husein/ssd3/postfilter-stt/part1'
X, Y = [], []

for key in tqdm(sorted(list(d.keys()))):
    values = {d_['k']: d_ for d_ in d[key]}
    sorted_k = sorted(values.keys())
    temp = []
    temp_text = []
    idx = []
    lens = 0
    last = None
    
    for k in sorted_k:
        y, l = read(values[k]['file'])
        t = values[k]['cleaned']
        ori_t = values[k]['text']

        if len(re.findall(r'\d+', t)):
            continue

        if len(t.strip()) < 2:
            continue
            
        if len(set(ori_t) - printable):
            continue

        if last is None:
            temp.append(y)
            idx.append(k)
            temp_text.append(t)
            lens += l

        elif k - last != 1 or (lens + l) >= 12:
            audio_path = os.path.join(directory, f'{index}.mp3')
            a = np.concatenate(temp, axis = 0)
            torchaudio.save(audio_path, 
                            torch.tensor(a.astype('float32')).unsqueeze(0), 
                            16000, format='mp3')
            X.append(audio_path)
            Y.append(' '.join(temp_text).strip())
            index += 1
            
            temp = [y]
            lens = l
            idx = [k]
            temp_text = [t]
        else:
            lens += l
            temp.append(y)
            idx.append(k)
            temp_text.append(t)

        last = k

    if len(temp):
        audio_path = os.path.join(directory, f'{index}.mp3')
        a = np.concatenate(temp, axis = 0)
        torchaudio.save(audio_path, 
                            torch.tensor(a.astype('float32')).unsqueeze(0), 
                            16000, format='mp3')
        X.append(audio_path)
        Y.append(' '.join(temp_text).strip())
        index += 1

  6%|█▉                                | 14013/239079 [42:07<8:35:14,  7.28it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 34%|██████████▋                     | 80285/239079 [3:27:15<3:43:00, 11.87it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 37%|███████████▉                    | 88818/239079 [3:47:59<6:15:54,  6.66it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order t

In [17]:
len(X)

707960

In [18]:
total_lens = []
for f in tqdm(X):
    audio = reader.decode_example(reader.encode_example(f))['array']
    total_lens.append(len(audio) / sr)

 72%|████████████████████████▍         | 509472/707960 [15:37<06:07, 540.66it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [19]:
import IPython.display as ipd
ipd.Audio(audio, rate = 16000)

In [20]:
Y[-1]

'do you have a lot of friends who were not of your race let s start with ming an'

In [21]:
sum(total_lens) / 60 / 60

1667.8041100007201

In [22]:
with open('postfilter-part-2023-06-19.json', 'w') as fopen:
    json.dump({'X': X, 'Y': Y}, fopen)